In [55]:
import os
import gc
from tqdm import tqdm # track progress
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from natsort import natsorted

def occlude_face_images(input_path_img, input_path_annotations, output_path, batch_size=0, mem_clean_after=100):
    img_list = os.listdir(input_path_img) # list of all images in folder
    img_list = natsorted(img_list)

    batch_counter = 0
    # Create a tqdm progress bar
    total_images = min(len(img_list), batch_size if batch_size != 0 else batch_size + 1)
    progress_bar = tqdm(total=total_images, desc='Processing Images', unit='images')
    
    # Create checkpoint file
    chkpt = os.path.join(output_path, "checkpoint.txt")
    if os.path.exists(chkpt):
        with open(chkpt, 'r') as f:
            last_processed = f.readline().strip()
    else:
        last_processed = ''
    
    # Get index of last processed file
    last_processed_index = 0 if last_processed == '' else img_list.index(last_processed)
    
    for i in range(last_processed_index + 1, len(img_list)):
        if batch_counter == batch_size and batch_counter > 0:
            break
        
        filename = os.path.splitext(img_list[i])[0] # splits filename from extension ['filename', '.ext']
        ext = os.path.splitext(img_list[i])[1]

        # Load facial landmarks from the npy file
        l = np.load(os.path.join(input_path_annotations, filename + '_lnd.npy'))

        # Convert landmarks to a 2D array with shape (68, 2)
        l = l.reshape((68, 2))

        # Load corresponding face image
        img = mpimg.imread(os.path.join(input_path_img, img_list[i]))

        # Get original image dimensions
        original_height, original_width, _ = img.shape

        # Create a figure with the same dimensions as the original image
        fig = plt.figure(figsize=(original_width / 100, original_height / 100), dpi=100)
        ax = fig.add_axes([0, 0, 1, 1])

        ax.imshow(img)
        ax.set_frame_on(False)
        ax.axis('off')

        # Calculate the distance between facial landmark 0 and facial landmark 16
        # norm = vector length, euclidean distance = distance
        w = np.linalg.norm(l[0] - l[16])

        # meta quest 2 aspect ratio (width/height)
        ar = 224/105

        # Calculate the height of the rectangle
        h = w / (ar)

        # Calculate the center point between facial landmark 36 and facial landmark 45 (eyes)
        m_ocd = l[36] + (l[45] - l[36]) / 2

        # Calculate the angle between facial landmark 36 and facial landmark 45
        a = l[45, 0] - l[36, 0] # Ankathete, x-axis
        b = l[45, 1] - l[36, 1] # Gegenkathete, y-axis
        alpha = np.arctan2(b, a)

        # Calculate the corner point of the rectangle
        R_a = m_ocd - w / 2 * np.array([np.cos(alpha), np.sin(alpha)]) - h / 2 * np.array([-np.sin(alpha), np.cos(alpha)])

        # Create and add the rectangle patch to the plot
        rect = patches.Rectangle((R_a[0], R_a[1]), w, h, angle=np.degrees(alpha), facecolor='black')
        ax.add_patch(rect)

        # Save File
        plt.savefig(output_path + filename + '_occ.jpg', format='jpg', bbox_inches='tight', pad_inches=0)

        # Remove Image from Memory
        plt.close(fig)
        
        #if batch_counter % mem_clean_after == 0: 
        #    del(img)
        #    del(fig)
        #    gc.collect()
        
        # Update the checkpoint after processing each file
        with open(chkpt, 'w') as f:
            f.write(filename + ext)
        batch_counter += 1
        progress_bar.update(1) # Update progress bar
    
    # Close progress bar when done
    progress_bar.close()

In [51]:
# dataset paths

#affnet_img = '../AffectNet/train_set/images/'
#affnet_anno = '../AffectNet/train_set/annotations/'
#output_path = '../AffectNet_Occluded/train_set/images/'

#affnet_img = r'C:\Users\LEAND\Coding\_FER\AffectNet\train_set\images\\'
#affnet_anno = r'C:\Users\LEAND\Coding\_FER\AffectNet\train_set\annotations\\'
#output_path = r'C:\Users\LEAND\Coding\_FER\AffectNet_Occluded\train_set\images\\'

affnet_img = r'C:\Users\LEAND\Coding\_FER\AffectNet\val_set\images\\'
affnet_anno = r'C:\Users\LEAND\Coding\_FER\AffectNet\val_set\annotations\\'
output_path = r'C:\Users\LEAND\Coding\_FER\AffectNet_Occluded\val_set\images\\'

In [27]:
#%%timeit
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)
gc.collect()

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [06:59<00:00, 23.81images/s]


930

In [28]:
gc.collect()

21700000

In [29]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:00<00:00, 23.76images/s]


In [30]:
gc.collect()

21702409

In [31]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:00<00:00, 23.75images/s]


In [32]:
gc.collect()

21701895

In [33]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:01<00:00, 23.73images/s]


In [34]:
gc.collect()

21701675

In [35]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:09<00:00, 23.30images/s]


In [36]:
gc.collect()

21702109

In [37]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:07<00:00, 23.37images/s]


In [38]:
gc.collect()

21702245

In [39]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:09<00:00, 23.27images/s]


In [40]:
gc.collect()

21702146

In [41]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:11<00:00, 23.17images/s]


In [42]:
gc.collect()

21701884

In [43]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=10000)

Processing Images: 100%|█████████████████████████████████████████████████████| 10000/10000 [07:19<00:00, 22.77images/s]


In [44]:
gc.collect()

21702440

In [52]:
occlude_face_images(affnet_img, affnet_anno, output_path, batch_size=4000)

Processing Images: 100%|██████████████████████████████████████████████████████▉| 3998/3999 [02:35<00:00, 25.75images/s]


In [53]:
gc.collect()

8676475

In [ ]:
287651